### Bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.stattools import ccf

### Dataframe com o RESAMPLE de 5 MINUTOS

In [2]:
caminho_arquivos_pkl_29_01_and_20_05 = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_5_minutos.pkl'
df_resample_5_min = pd.read_pickle(caminho_arquivos_pkl_29_01_and_20_05)

### Excluir o 6 de maio por ser uma segunda-feira 

In [3]:
df_resample_5_min# Filtrar os dados para excluir o dia 20 de maio
df_resample_5_min = df_resample_5_min[df_resample_5_min.index.date != pd.to_datetime('2024-05-20').date()]

### GUARDAR NA PASTA "graficos_medias_3_meses_resample_5_minutos_cada_variavel" as médias mensais de cada variável em cada 5 min

In [4]:
# DataFrame com os dados resample de 5 minutos
# Vamos supor que o DataFrame se chama df_resample_5_min e tem as colunas mencionadas
# Diretório onde as imagens serão salvas
output_directory = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\graficos_medias_3_meses_resample_5_minutos_cada_variavel'
# Agrupar por hora do dia e calcular a média de todas as colunas
df_hourly_mean = df_resample_5_min.groupby(df_resample_5_min.index.time).mean()

# Lista das colunas de interesse
columns_of_interest = ['G2_Gerador_Potência',
    'G2_Oleo_Motor_Temp', 'G2_Oleo_Motor_Press',
    'G2_Carga_Temperature', 'G2_Aumento_Pressão_Valvula_Real',
    'G2_Gerador_Corrente_Média', 'G2_Gerador_Tensão_Média',
    'G2_Gerador_Potência_Reativa', 'G2_Gerador_Potência_Aparente',
    'G2_Gerador_frequencia', 'G2_Gerador_Fator_de_Potência',
    'G2_Acelerador_Valv_Posição', 'G2_Jacket_Water_pressão',
    'G2_Jacket_Water_Temp', 'G2_Cilindro_EX_GAS_TEMP_Média',
    'g2_Turbocharger_Bypass_Posição', 'G2_Saída_Ativa_total',
    'G2_Saída_Reativa_total', 'G2_LAMBDA_TECJET', 'G2_Gerador_Corrente_L1',
    'G2_Gerador_Voltagem_L1_N', 'G2_Gerador_enrolamento_temp_L1',
    'G2_Gerador_enrolamento_temp_L2', 'G2_Gerador_enrolamento_temp_L3',
    'G2_Gerador_Extremidade_acionamento_rolamento',
    'G2_Gerador_Bearing_nondriveend', 'G2_Gerador_Corrente_L2',
    'G2_Gerador_Voltagem_L2_L3', 'G2_Gerador_Voltagem_L2_N',
    'G2_Gerador_Corrente_N', 'G2_Excitação_Voltagem',
    'G2_Gerador_Corrente_L3', 'G2_Gerador_Voltagem_L3_L1',
    'G2_Gerador_Voltagem_L3_N'
]

# Criar lista de horários para os ticks do eixo x (como strings)
time_index_str = [time.strftime("%H:%M") for time in pd.date_range("00:00", "23:55", freq="5min").time]

# Plotar um gráfico para cada variável
for column in columns_of_interest:
    plt.figure(figsize=(20, 5))  # Define o tamanho da figura
    plt.plot(np.arange(len(df_hourly_mean)), df_hourly_mean[column], label=column)
    plt.xlabel('Tempo do Dia')
    plt.ylabel('Média')
    plt.title(f'Média de {column} a cada 5 minutos')
    plt.xticks(np.arange(0, len(df_hourly_mean), step=12), time_index_str[::12], rotation=45)  # Define os ticks do eixo x
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)  # Adiciona grid a cada 15 minutos

    # Salvar a imagem no diretório especificado
    output_filename = os.path.join(output_directory, f'medias_variaveis_3_meses_cada_5_minutos_{column}.jpg')
    plt.savefig(output_filename)
    plt.close()  # Fechar a figura para liberar memória

### GUARDAR em "graficos_medias_3_meses_resample_5_minutos_cada_variavel_com_Target" as médias mensais de cada variável em cada 5 min, com a variavel Target

In [6]:
# Supondo que você já tenha o DataFrame df_resample_5_min
# Diretório onde as imagens serão salvas
output_directory = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\graficos_medias_3_meses_resample_5_minutos_cada_variavel_com_Target'
# Agrupar por hora do dia e calcular a média de todas as colunas
df_hourly_mean = df_resample_5_min.groupby(df_resample_5_min.index.time).mean()

# Criar lista de horários para os ticks do eixo x (como strings)
time_index_str = [time.strftime("%H:%M") for time in pd.date_range("00:00", "23:55", freq="5min").time]

# Combinar a potência gerada com cada outra variável em gráficos separados
for column in columns_of_interest:
    if column != 'G2_Gerador_Potência':
        fig, ax1 = plt.subplots(figsize=(20, 5))

        color = 'tab:blue'
        ax1.set_xlabel('Tempo do Dia')
        ax1.set_ylabel('G2_Gerador_Potência', color=color)
        ax1.plot(time_index_str, df_hourly_mean['G2_Gerador_Potência'], color=color)
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()
        color = 'tab:red'
        ax2.set_ylabel(column, color=color)
        ax2.plot(time_index_str, df_hourly_mean[column], color=color)
        ax2.tick_params(axis='y', labelcolor=color)

        plt.title(f'G2_Gerador_Potência e {column}')
        plt.xticks(np.arange(0, len(df_hourly_mean), step=12), time_index_str[::12], rotation=45) 
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        fig.tight_layout()
        
        # Salvar a imagem no diretório especificado
        output_filename = os.path.join(output_directory, f'medias_variaveis_3_meses_cada_5_minutos_com_Target_{column}.jpg')
        plt.savefig(output_filename)
        plt.close()  # Fechar a figura para liberar memória

### Mostrar o dataframe que tem as médias mensais a cada 5 minutos

In [7]:
df_hourly_mean

,G2_Gerador_Potência,G2_Oleo_Motor_Temp,G2_Oleo_Motor_Press,G2_Carga_Temperature,G2_Aumento_Pressão_Valvula_Real,G2_Gerador_Corrente_Média,G2_Gerador_Tensão_Média,G2_Gerador_Potência_Reativa,G2_Gerador_Potência_Aparente,G2_Gerador_frequencia,...,G2_Gerador_Extremidade_acionamento_rolamento,G2_Gerador_Bearing_nondriveend,G2_Gerador_Corrente_L2,G2_Gerador_Voltagem_L2_L3,G2_Gerador_Voltagem_L2_N,G2_Gerador_Corrente_N,G2_Excitação_Voltagem,G2_Gerador_Corrente_L3,G2_Gerador_Voltagem_L3_L1,G2_Gerador_Voltagem_L3_N
00:00:00,542.918919,58.450450,2.727928,51.531081,1.345833,50.261261,4121.644144,84.789826,558.664414,32.367568,...,32.604054,23.318468,50.495495,4121.554054,2393.204955,0.000000,7.469820,53.006757,4115.409910,2383.171171
00:05:00,0.000000,56.682883,0.390293,56.084910,0.994054,0.000000,0.000000,0.000000,0.000000,0.000000,...,32.030631,26.350676,0.000000,0.000000,0.000000,0.000000,0.210811,0.000000,0.000000,0.000000
00:10:00,0.000000,55.592793,0.415045,57.595045,0.994369,0.000000,0.000000,0.000000,0.000000,0.000000,...,31.736036,28.220721,0.000000,0.000000,0.000000,0.000000,0.209910,0.000000,0.000000,0.000000
00:15:00,0.000000,56.998198,0.407973,57.816667,0.994640,0.000000,0.000000,0.000000,0.000000,0.000000,...,31.518919,29.220946,0.000000,0.000000,0.000000,0.000000,0.209009,0.000000,0.000000,0.000000
00:20:00,0.000000,57.439414,0.407658,58.003829,0.994730,0.000000,0.000000,0.000000,0.000000,0.000000,...,31.348874,29.918018,0.000000,0.000000,0.000000,0.000000,0.207658,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23:35:00,2829.222289,59.570759,2.646009,53.237587,4.284129,256.429007,4171.851969,700.117430,2915.121733,32.122811,...,32.872953,23.478354,256.373055,4171.199473,2440.130226,0.006696,17.251699,259.874129,4164.496188,2429.192141
23:40:00,2829.144551,59.513170,2.646345,53.202345,4.284397,256.431368,4173.966679,704.004169,2916.223860,32.133539,...,32.838054,23.475387,256.353243,4173.377932,2441.312581,0.026786,17.265512,259.842620,4166.715550,2430.432552
23:45:00,2829.256546,59.458929,2.646391,53.211523,4.284821,256.547569,4171.449693,707.040461,2916.654112,32.125293,...,32.798691,23.417510,256.552033,4170.849397,2439.800739,0.026786,17.279772,260.014237,4164.365716,2429.058974
23:50:00,2829.419881,59.416071,2.646615,53.236772,4.284531,256.482966,4171.731814,702.962914,2915.963828,32.125082,...,32.757542,23.407624,256.500823,4171.202558,2440.041130,0.000000,17.255192,259.964871,4164.531506,2429.254592


### Guardar o dataframe que tem as médias mensais a cada 5 minutos

In [8]:
# Diretório onde você deseja salvar os arquivos
diretorio = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel'

# Nome do arquivo
nome_arquivo = 'medias_resample_5_min_mensais'

# Salvar em formato pickle (.pkl)
caminho_pkl = os.path.join(diretorio, f'{nome_arquivo}.pkl')
df_hourly_mean.to_pickle(caminho_pkl)

# Salvar em formato CSV (.csv)
caminho_csv = os.path.join(diretorio, f'{nome_arquivo}.csv')
df_hourly_mean.to_csv(caminho_csv)

### GRAFICOS DAS MÉDIAS SEMANAIS - SAZONALIDADE SEMANAL

In [9]:
# Extrair as semanas do ano
semanas = df_resample_5_min.index.to_period('W')

# Criar um DataFrame vazio para armazenar a média semanal para cada variável
df_final = pd.DataFrame(columns=df_resample_5_min.columns)

# Iterar sobre as semanas
for semana in semanas.unique():
    # Filtrar o DataFrame original para os dados da semana atual
    df_semana = df_resample_5_min.loc[semana.start_time:semana.end_time]
    
    # Iterar sobre cada período de 5 minutos dentro da semana
    for i in range(12 * 24):
        # Calcular a média para cada variável no período de 5 minutos atual
        media_periodo = df_semana.iloc[i::12*24].mean()
        # Adicionar a média ao DataFrame final com o índice apropriado
        df_final.loc[semana.start_time + pd.Timedelta(minutes=5*i)] = media_periodo

# Agora df_final contém a média semanal para cada variável ao longo de cada semana, para cada período de 5 minutos

### Mostrar o dataframe que tem as médias semanais a cada 5 minutos (12x24 periodos de 5 minutos por dia *16 semanas)

In [11]:
df_final

,G2_Gerador_Potência,G2_Oleo_Motor_Temp,G2_Oleo_Motor_Press,G2_Carga_Temperature,G2_Aumento_Pressão_Valvula_Real,G2_Gerador_Corrente_Média,G2_Gerador_Tensão_Média,G2_Gerador_Potência_Reativa,G2_Gerador_Potência_Aparente,G2_Gerador_frequencia,...,G2_Gerador_Extremidade_acionamento_rolamento,G2_Gerador_Bearing_nondriveend,G2_Gerador_Corrente_L2,G2_Gerador_Voltagem_L2_L3,G2_Gerador_Voltagem_L2_N,G2_Gerador_Corrente_N,G2_Excitação_Voltagem,G2_Gerador_Corrente_L3,G2_Gerador_Voltagem_L3_L1,G2_Gerador_Voltagem_L3_N
2024-01-29 00:00:00,3141.785714,64.357143,2.916786,53.864286,4.677143,283.654762,4656.142857,777.571429,3237.571429,35.685714,...,34.585714,25.157143,282.952381,4656.214286,2722.107143,0.0,19.369048,286.714286,4650.857143,2712.357143
2024-01-29 00:05:00,3142.971429,64.405844,2.916429,53.917857,4.676429,283.000000,4663.857143,789.214286,3240.214286,35.700000,...,34.742857,25.328571,282.857143,4663.464286,2726.167208,0.0,19.425000,286.642857,4658.452381,2716.571429
2024-01-29 00:10:00,3139.357143,64.200000,2.918571,53.878571,4.674286,282.857143,4666.857143,791.714286,3238.035714,35.714286,...,34.942857,25.557143,282.214286,4666.285714,2727.571429,0.0,19.471429,285.928571,4661.428571,2718.285714
2024-01-29 00:15:00,3144.142857,64.057143,2.919286,53.950000,4.684286,283.428571,4656.000000,772.714286,3238.500000,35.700000,...,35.139286,25.714286,283.000000,4655.714286,2721.714286,0.0,19.410714,286.571429,4651.142857,2712.607143
2024-01-29 00:20:00,3141.142857,64.046429,2.920000,53.950000,4.676071,283.428571,4665.571429,793.142857,3239.571429,35.714286,...,35.271429,25.957143,282.642857,4665.285714,2726.857143,0.0,19.542857,286.000000,4660.571429,2717.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13 23:35:00,3145.285714,62.392857,2.919643,54.571429,4.726071,285.285714,4630.428571,775.714286,3240.285714,35.685714,...,32.900000,22.400000,285.271429,4629.973810,2708.857143,0.0,18.928571,289.142857,4622.642857,2696.854762
2024-05-13 23:40:00,3144.285714,62.400000,2.921429,54.617857,4.727143,284.857143,4639.035714,780.107143,3239.928571,35.714286,...,32.885714,22.550000,284.428571,4638.714286,2713.285714,0.0,18.928571,288.428571,4631.285714,2701.857143
2024-05-13 23:45:00,3144.571429,62.357143,2.920000,54.585714,4.727143,284.714286,4641.857143,784.500000,3241.857143,35.685714,...,32.828571,22.328571,284.285714,4641.285714,2714.571429,0.0,18.978571,288.285714,4634.142857,2703.285714
2024-05-13 23:50:00,3143.642857,62.328571,2.920000,54.614286,4.725714,284.571429,4639.285714,781.785714,3240.642857,35.685714,...,32.771429,22.357143,284.571429,4639.000000,2713.142857,0.0,18.942857,288.571429,4631.857143,2702.071429


### Guardar o dataframe que tem as médias semanais a cada 5 minutos

In [12]:
# Diretório onde você deseja salvar os arquivos
diretorio = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel'

# Nome do arquivo
nome_arquivo = 'medias_resample_5_min_semanais'

# Salvar em formato pickle (.pkl)
caminho_pkl = os.path.join(diretorio, f'{nome_arquivo}.pkl')
df_final.to_pickle(caminho_pkl)

# Salvar em formato CSV (.csv)
caminho_csv = os.path.join(diretorio, f'{nome_arquivo}.csv')
df_final.to_csv(caminho_csv)

### GUARDAR NA PASTA "graficos_medias_semanais_resample_5_minutos_cada_variavel" as médias semanais de cada variável em cada 5 min

In [21]:
# Diretório de saída
output_dir = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\graficos_medias_semanais_resample_5_minutos_cada_variavel'

# Calcular o mínimo dos mínimos e o máximo dos máximos para cada variável ao longo de todas as semanas
minimos = df_final.min() - 50
maximos = df_final.max() + 50

# Iterar sobre cada variável
for column in df_final.columns:
    # Iterar sobre cada dia
    for date in df_final.index.floor('D').unique():
        # Filtrar o DataFrame final para os dados do dia atual e variável atual
        df_dia = df_final.loc[date:date + pd.Timedelta(days=1)]

        # Criar a figura e os eixos
        fig, ax1 = plt.subplots(figsize=(20, 5))

        # Plotar a potência gerada no eixo yy do lado esquerdo
        ax1.plot(df_dia.index, df_dia['G2_Gerador_Potência'], label='G2_Gerador_Potência', linestyle='-', color='blue')
        ax1.set_ylabel('G2_Gerador_Potência', color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.yaxis.label.set_color('blue')

        # Plotar a variável no eixo yy do lado direito
        ax2 = ax1.twinx()
        ax2.plot(df_dia.index, df_dia[column], label=f'{column}', color='red')
        ax2.set_ylabel('Média', color='red')
        ax2.tick_params(axis='y', labelcolor='red')
        ax2.yaxis.label.set_color('red')

        # Adicionar título e legenda
        plt.title(f'Média horária de {column} e G2_Gerador_Potência para o dia {date.strftime("%d/%m/%Y")}')
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')

        # Definir limites dos eixos
        ax1.set_ylim(minimos['G2_Gerador_Potência'], maximos['G2_Gerador_Potência'])
        ax2.set_ylim(minimos[column], maximos[column])

        # Formatar ticks do eixo x
        plt.xticks(rotation=45)
        ax1.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Hh%M'))

        # Adicionar grid na vertical a cada hora
        ax1.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5)
        
        # Salvar o gráfico
        filename = f'{output_dir}/{column}_{date.strftime("%Y_%m_%d")}.png'
        plt.savefig(filename, dpi=300)
        
        # Fechar o plot para liberar memória
        plt.close(fig)

In [16]:
df_final

,G2_Gerador_Potência,G2_Oleo_Motor_Temp,G2_Oleo_Motor_Press,G2_Carga_Temperature,G2_Aumento_Pressão_Valvula_Real,G2_Gerador_Corrente_Média,G2_Gerador_Tensão_Média,G2_Gerador_Potência_Reativa,G2_Gerador_Potência_Aparente,G2_Gerador_frequencia,...,G2_Gerador_Extremidade_acionamento_rolamento,G2_Gerador_Bearing_nondriveend,G2_Gerador_Corrente_L2,G2_Gerador_Voltagem_L2_L3,G2_Gerador_Voltagem_L2_N,G2_Gerador_Corrente_N,G2_Excitação_Voltagem,G2_Gerador_Corrente_L3,G2_Gerador_Voltagem_L3_L1,G2_Gerador_Voltagem_L3_N
2024-01-29 00:00:00,3141.785714,64.357143,2.916786,53.864286,4.677143,283.654762,4656.142857,777.571429,3237.571429,35.685714,...,34.585714,25.157143,282.952381,4656.214286,2722.107143,0.0,19.369048,286.714286,4650.857143,2712.357143
2024-01-29 00:05:00,3142.971429,64.405844,2.916429,53.917857,4.676429,283.000000,4663.857143,789.214286,3240.214286,35.700000,...,34.742857,25.328571,282.857143,4663.464286,2726.167208,0.0,19.425000,286.642857,4658.452381,2716.571429
2024-01-29 00:10:00,3139.357143,64.200000,2.918571,53.878571,4.674286,282.857143,4666.857143,791.714286,3238.035714,35.714286,...,34.942857,25.557143,282.214286,4666.285714,2727.571429,0.0,19.471429,285.928571,4661.428571,2718.285714
2024-01-29 00:15:00,3144.142857,64.057143,2.919286,53.950000,4.684286,283.428571,4656.000000,772.714286,3238.500000,35.700000,...,35.139286,25.714286,283.000000,4655.714286,2721.714286,0.0,19.410714,286.571429,4651.142857,2712.607143
2024-01-29 00:20:00,3141.142857,64.046429,2.920000,53.950000,4.676071,283.428571,4665.571429,793.142857,3239.571429,35.714286,...,35.271429,25.957143,282.642857,4665.285714,2726.857143,0.0,19.542857,286.000000,4660.571429,2717.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13 23:35:00,3145.285714,62.392857,2.919643,54.571429,4.726071,285.285714,4630.428571,775.714286,3240.285714,35.685714,...,32.900000,22.400000,285.271429,4629.973810,2708.857143,0.0,18.928571,289.142857,4622.642857,2696.854762
2024-05-13 23:40:00,3144.285714,62.400000,2.921429,54.617857,4.727143,284.857143,4639.035714,780.107143,3239.928571,35.714286,...,32.885714,22.550000,284.428571,4638.714286,2713.285714,0.0,18.928571,288.428571,4631.285714,2701.857143
2024-05-13 23:45:00,3144.571429,62.357143,2.920000,54.585714,4.727143,284.714286,4641.857143,784.500000,3241.857143,35.685714,...,32.828571,22.328571,284.285714,4641.285714,2714.571429,0.0,18.978571,288.285714,4634.142857,2703.285714
2024-05-13 23:50:00,3143.642857,62.328571,2.920000,54.614286,4.725714,284.571429,4639.285714,781.785714,3240.642857,35.685714,...,32.771429,22.357143,284.571429,4639.000000,2713.142857,0.0,18.942857,288.571429,4631.857143,2702.071429


### GUARDAR em "graficos_medias_semanais_resample_5_minutos_cada_variavel_com_Target" as médias semanais de cada variável em cada 5 min, com a variavel Target

In [20]:
# Diretório de saída
output_dir = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\graficos_medias_semanais_resample_5_minutos_cada_variavel_com_Target'

# Calcular o mínimo dos mínimos e o máximo dos máximos para cada variável ao longo de todas as semanas
minimos = df_final.min() - 50
maximos = df_final.max() + 50

# Iterar sobre cada variável
for column in df_final.columns:
    # Iterar sobre cada dia
    for date in df_final.index.floor('D').unique():
        # Filtrar o DataFrame final para os dados do dia atual e variável atual
        df_dia = df_final.loc[date:date + pd.Timedelta(days=1)]

        # Criar a figura e os eixos
        fig, ax1 = plt.subplots(figsize=(20, 5))

        # Plotar a potência gerada no eixo yy do lado esquerdo
        ax1.plot(df_dia.index, df_dia['G2_Gerador_Potência'], label='G2_Gerador_Potência', linestyle='-', color='blue')
        ax1.set_ylabel('G2_Gerador_Potência', color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.yaxis.label.set_color('blue')

        # Plotar a variável no eixo yy do lado direito
        ax2 = ax1.twinx()
        ax2.plot(df_dia.index, df_dia[column], label=f'{column}', color='red')
        ax2.set_ylabel('Média', color='red')
        ax2.tick_params(axis='y', labelcolor='red')
        ax2.yaxis.label.set_color('red')

        # Adicionar título e legenda
        plt.title(f'Média horária de {column} e G2_Gerador_Potência para o dia {date.strftime("%d/%m/%Y")}')
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')

        # Definir limites dos eixos
        ax1.set_ylim(minimos['G2_Gerador_Potência'], maximos['G2_Gerador_Potência'])
        ax2.set_ylim(minimos[column], maximos[column])

        # Formatar ticks do eixo x
        plt.xticks(rotation=45)
        ax1.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Hh%M'))

        # Adicionar grid na vertical a cada hora
        ax1.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5)
        
        # Salvar o gráfico
        filename = f'{output_dir}/{column}_{date.strftime("%Y_%m_%d")}.png'
        plt.savefig(filename, dpi=300)
        
        # Fechar o plot para liberar memória
        plt.close(fig)

